<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/assignment7_wordembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload

In [2]:
import os 
os.environ['OMP_NUM_THREADS'] = '64'

In [3]:
import sys
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    !pip install --upgrade gensim -qq
    !pip install -U spacy -qq


    basepath = '/content/drive/MyDrive/NLP/assignment7'
    sys.path.append('/content/drive/MyDrive/NLP/custom-functions')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import random
import spacy
import textwrap
import joblib
from pathlib import Path
import numpy as np
import gensim
import gensim.downloader as loader
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import itertools

In [5]:
import os
os.cpu_count()

2

In [6]:
print(f'spacy-version: {spacy.__version__}  || gensim-version{ gensim.__version__}')

spacy-version: 3.5.2  || gensim-version4.3.1


In [7]:
!python3 --version

Python 3.9.16


In [8]:
if gensim.models.doc2vec.FAST_VERSION > -1:
    print("Fast gensim version is being used.")
else:
    print("Slow gensim version is being used.")

Fast gensim version is being used.


In [9]:
folder = Path(basepath)/'WordEmbeddings'

In [10]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.9/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/NLP/custom-functions']

In [17]:
stackExchange_cleaned_text_file = folder / 'df_raw_small_hw.joblib'    #Folder path was set for accessing the file

In [18]:
#Load the file 
stackExchange_cleaned_text = joblib.load(stackExchange_cleaned_text_file)

FileNotFoundError: ignored

In [15]:
for text in imdb_unsup_cleaned_text_file:
  print(text)

TypeError: ignored